In [1]:
# importing the modules

import numpy as np
import pandas as pd
import math
import random


train_path='train.csv'
test_path='test.csv'
train_data=pd.read_csv(train_path)
test_data=pd.read_csv(test_path)


m_1=len(train_data['PassengerId'])
n_1=7                            #no(6) extracted parameters from train.csv #(now 7)was 6 Age added
train_array=np.zeros((m_1,n_1),dtype=None)  #extracted array for training with 5 parameters and 1 labels

max_age=test_data['Age'].max(skipna=True)
min_age=test_data['Age'].min(skipna=True)
max_fare=test_data['Fare'].max(skipna=True)
min_fare=test_data['Fare'].max(skipna=True) 
#convert the to a matrix of (m_1,n_1)
for i in range(m_1):
    survived=train_data['Survived'][i]
    train_array[i,0]=int(survived)

    pclass=train_data['Pclass'][i]
    train_array[i,1]=float(pclass)

    sex=train_data['Sex'][i]
    if sex=='male':
        train_array[i,2]=1
    elif sex=='female':
        train_array[i,2]=0
    
    sib=train_data['SibSp'][i]
    train_array[i,3]=float(sib)

    parch=train_data['Parch'][i]
    train_array[i,4]=float(parch)

    fare=train_data['Fare'][i]
    if math.isnan(fare):
        fare=random.randint(np.round(min_fare),np.round(max_fare))
    train_array[i,5]=float(fare)

    age=train_data['Age'][i]
    if math.isnan(age):
        age=random.random()*(max_age-min_age)
    train_array[i,6]=float(age)



m_2=len(test_data['PassengerId']) #number of entry
n_2=6                          #number(5) of parameters without passengerID # was 5 .Age added
test_array=np.zeros((m_2,n_2),dtype=None)   #array for test from test
passenger_id=np.zeros((1,m_2),dtype=int)   #array for passenger_id

max_age=test_data['Age'].max(skipna=True)
min_age=test_data['Age'].min(skipna=True)
max_fare=test_data['Fare'].max(skipna=True)
min_fare=test_data['Fare'].max(skipna=True)
#convert data into (m_2,n_2) array
for i in range(m_2):
    
    pclass=test_data['Pclass'][i]
    test_array[i,0]=float(pclass)

    sex=test_data['Sex'][i]
    if sex=='male':
        test_array[i,1]=1
    elif sex=='female':
        test_array[i,1]=0
    
    sib=test_data['SibSp'][i]
    test_array[i,2]=float(sib)

    parch=test_data['Parch'][i]
    test_array[i,3]=float(parch)

    fare=test_data['Fare'][i]
    if math.isnan(fare):
        fare=random.randint(np.round(min_fare),np.round(max_fare))
    test_array[i,4]=float(fare)

    age=test_data['Age'][i]
    if math.isnan(age):
        age=random.randint(np.round(min_age),np.round(max_age))
    test_array[i,5]=float(age)

    Id=test_data['PassengerId'][i]
    passenger_id[0,i]=int(Id)

 
#Y=ex_array[:,0:1]
input_labels=train_array[:,0:1].T # train lables 
inp_array=train_array[:,1:]      #input array after pre-processing

def normalization(ex_array):
    m=ex_array.shape[0]

    mu=(1/m)*np.sum(ex_array,axis=0,keepdims=True)
    sig=(1/m)*np.sum(ex_array**2,axis=0,keepdims=True)
    ex_array=(ex_array-mu)/(sig+1e-11)

    return ex_array


train_array1=normalization(inp_array)
test_array1=normalization(test_array)

   
input_data=train_array1[:].T #input data in (n,m) ,(n=5,m=891)
test_data1=test_array[:].T  #test data eligble for test 
#labels in (1,m)

F1=pd.DataFrame(input_data)
F1.to_csv('Newdata1.csv',index=False)




In [2]:

n=input_data.shape[0]
layers_dims=(n,10,1)  #2 layer NN with 10 neurons in hidden layer
def param_intalization(layers_dims):
    n1,n2,n3=layers_dims
    W1=np.random.randn(n2,n1)*np.sqrt(2/n1)
    b1=np.zeros((n2,1))
    W2=np.random.randn(n3,n2)*np.sqrt(2/n2)
    b2=np.zeros((n3,1))

    assert(W1.shape==(n2,n1))
    assert(W2.shape==(n3,n2))
    assert(b1.shape==(n2,1))
    assert(b2.shape==(n3,1))

    param={
        'W1':W1,
        'b1':b1,
        'W2':W2,
        'b2':b2
    }
    return param



#linear activation (Z=W.X+b) #z=(2,m)
def linear_forward(A_prev,W,b):
    
    Z=np.dot(W,A_prev)+b

    assert(Z.shape==(W.shape[0],A_prev.shape[1]))

    cache=(A_prev,W,b)

    return Z,cache

#Z=linear_forward(input_data,params['W1'],params['b1'])

#print(Z.shape)

#sigmoid funciton
def sigmoid(Z):
    A=1/(1+np.exp(-Z))

    assert(A.shape==Z.shape)
    activation_cache=(Z,A)

    return A, activation_cache

def relu(Z):

    A=np.maximum(0,Z)

    assert(A.shape==Z.shape)
    activation_cache=(Z,A) 

    return A,activation_cache

def linear_activation_forward(A_prev,W,b,Activation):

    Z,linear_cache=linear_forward(A_prev,W,b)

    if Activation=='Sigmoid':
        A,activation_cache=sigmoid(Z)
    elif Activation=='relu':
        A,activation_cache=relu(Z)
    
    assert(A.shape==Z.shape)
    

    cache=(linear_cache,activation_cache)  ##activation_cache(Z,A),linear_cache(A_prev,W,b)

    return A, cache



def forward_prop(X,params):

    caches=[]

    W1=params['W1']
    b1=params['b1']
    W2=params['W2']
    b2=params['b2']
    #l=len(params)//2

    A1,cache=linear_activation_forward(X,W1,b1,'relu')
    caches.append(cache)
    A2,cache=linear_activation_forward(A1,W2,b2,'Sigmoid')
    caches.append(cache)

    AL=A2
    #print(AL.shape)
    assert(AL.shape==(1,X.shape[1]))

    
    return AL, caches
#AL=forward_prop(input_data,params)


def compute_cost(AL,Y,params,lamb=1.25):

    m=Y.shape[1]
    #l=lne(params)//2
    W1=params['W1']
    W2=params['W2']
    
    reg=np.sum(W1**2)+np.sum(W2**2)

    cost=-1/m*np.sum(Y*np.log(AL+1e-10)+ (1-Y)*np.log(1-AL-1e-10))+(lamb/(2*m))*reg

    '''j=Y*np.log(AL)+(1-Y)*np.log(1-AL)
    cost=(-1/m)*np.sum(j)'''

    cost = np.squeeze(cost)   #retruns as number not array
    assert(cost.shape == ())

    return cost

def linear_backward(dZ,linear_cache):
    A_prev,W,b=linear_cache
    m=A_prev.shape[1]
    
    dW=(1/m)*np.dot(dZ,A_prev.T)
    db=(1/m)*np.sum(dZ,axis=1,keepdims=True)
    dA_prev=np.dot(W.T,dZ)

    assert(dA_prev.shape==A_prev.shape)
    assert(dW.shape==W.shape)
    assert(db.shape==b.shape)

    return dA_prev, dW, db

def sigmoid_backward(dA, cache):
    """
    Implement the backward propagation for a single SIGMOID unit.

    Arguments:
    dA -- post-activation gradient, of any shape
    cache -- 'Z' where we store for computing backward propagation efficiently

    Returns:
    dZ -- Gradient of the cost with respect to Z
    """
    
    Z,A = cache
    
    dZ = dA * A * (1-A)
    
    assert (dZ.shape == Z.shape)
    
    return dZ

def relu_backward(dA, cache):
    """
    Implement the backward propagation for a single RELU unit.

    Arguments:
    dA -- post-activation gradient, of any shape
    cache -- 'Z' where we store for computing backward propagation efficiently

    Returns:
    dZ -- Gradient of the cost with respect to Z
    """
    
    Z,_ = cache
    dZ = np.array(dA) # just converting dz to a correct object.
    
    # When z <= 0, you should set dz to 0 as well. 
    dZ[Z<0]=0
    
    assert (dZ.shape == Z.shape)
    
    return dZ


def linear_activation_backword(dA,cache,activation):
    linear_cache,activation_cache=cache
    
    if activation=='relu':
        dZ=relu_backward(dA,activation_cache)
        dA_prev,dW,db=linear_backward(dZ,linear_cache)

    elif activation=='Sigmoid':
        dZ=sigmoid_backward(dA,activation_cache)
        dA_prev,dW,db=linear_backward(dZ,linear_cache)
    
    return dA_prev,dW,db




def model_back_prop(AL,Y,caches):

    grads={}
    #m=AL.shape[1]
    #Y=Y.reshape(AL.shape)

    dAL=- (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))

    #current_cache=caches[1]

    grads["dA1"],grads['dW2'],grads['db2']=linear_activation_backword(dAL,caches[1],activation='Sigmoid')
    grads['dA0'],grads['dW1'],grads['db1']=linear_activation_backword(grads['dA1'],caches[0],activation='relu')

    return grads


#update parameter

def update_params(X,params,grads,learning_rate=0.3,lamb=1.25):

    W1=params['W1']
    W2=params['W2']

    dW1=grads['dW1']
    db1=grads['db1']
    dW2=grads['dW2']
    db2=grads['db2']
    m=X.shape[1]

    params['W1']=params['W1']-learning_rate*(dW1+(lamb/m)*W1)
    params['b1']=params['b1']-learning_rate*db1
    params['W2']=params['W2']-learning_rate*(dW2+(lamb/m)*W2)
    params['b2']=params['b2']-learning_rate*db2

    return params


def optimize(X,Y,num_iterations,learning_rate=0.1,lamb=1.25):
    costs=[]
    n=X.shape[0]
    layers_dims=(n,10,1)
    params=param_intalization(layers_dims)
    


    for i in range(num_iterations):

        AL,caches=forward_prop(X,params)

        cost=compute_cost(AL,Y,params,lamb=1.25)
        #print(cost)
        grads=model_back_prop(AL,Y,caches)
        params=update_params(X,params,grads,learning_rate,lamb=1.25)

        if i%1000==0:
            costs.append(cost)
            print(cost)
        
    
    return params, costs



params,cost=optimize(input_data,input_labels,10000,0.3,1.25) 

def predict(X,params):
    #m=X.shape[1]
    #Y_prediction=np.zeros((1,m))
    AL,_=forward_prop(X,params)
    Y_prediction=np.round(AL)


    
    assert(Y_prediction.shape==(1,X.shape[1]))

    return Y_prediction


predictions=predict(input_data,params)
print('accuracy=='+str(np.mean(predictions==input_labels)*100))
#print(predictions)
predictions1=predict(test_data1,params)
print((predictions1[0]))
#passenger_ID=test_data['PassengerId']

test_ans_data=pd.read_csv('my_submissionk1.csv')

#test_ans=test_ans_data.to_dict()

test_ans=test_ans_data['Survived'].tolist()
print('accuracy_test=='+str(np.mean(predictions1==test_ans)*100))



predictions1=predictions1.tolist()
passenger_id=passenger_id.tolist()
data_f={
    'PassengerId':passenger_id[0],
    'Survived':predictions1[0]
}




0.7001546977033836
0.4481611079227099
0.44495210104045785
0.44378642180258315
0.4431985243111644
0.4429612011174917
0.4428592804000404
0.4427536583080067
0.4426405128175211
0.4426168676141016
accuracy==80.58361391694724
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

In [3]:
output = pd.DataFrame(data=data_f)

output.to_csv('my_submission.csv',index=False)